

# Pipeline to produce llm code samples based on "Human-Eval" dataset of opeanai

## Pip

### Basics

In [7]:
# Human-Eval by OpenAi for the problem dataset
!git clone "https://github.com/openai/human-eval.git"

# Accelerate for gpu computing
!pip install accelerate

SyntaxError: invalid syntax (3274864288.py, line 2)

### GPTQ model requirements

In [5]:
!pip install auto-gptq --extra-index-url https://huggingface.github.io/autogptq-index/whl/cu118/ 
!pip install optimum

SyntaxError: invalid syntax (1134316694.py, line 1)

### Phi2

In [ ]:
!pip install einops
!pip install auto-gptq
!pip install optimum

### Wizard-Coder

In [ ]:
!pip install auto_gptq
!pip install optimum

## Imports

### Basics

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import time
import sys
sys.path.insert(0,'/content/human-eval')
from human_eval.data import write_jsonl, read_problems
import locale
locale.getpreferredencoding = lambda: "UTF-8"

### Wizardcoder

In [ ]:
from transformers import AutoTokenizer, pipeline, logging
from auto_gptq import AutoGPTQForCausalLM, BaseQuantizeConfig
import argparse

## Model setup

All Models are quantized version of original models and used from "https://huggingface.co/TheBloke"

The models are lisensed by their original owners

Codellama: Meta (https://huggingface.co/codellama)
WizardCoder: WizardLM (https://huggingface.co/WizardLM)
Phi-2: Microsoft (https://huggingface.co/microsoft/phi-2)

What is GPTQ? - GPTQ is a post-training quantization method to compress LLMs and obtain nearly similiar accuracy

### Codellama-13b (GPTQ)

In [ ]:
model_path = "TheBloke/CodeLlama-13B-Instruct-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model,device_map="auto", trust_remote_code=True, revision="gptq-8bit-128g-actorder_True") # Biggest gptq model for codellama 13b

tokenizer = AutoTokenizer.from_pretrained("TheBloke/CodeLlama-13B-Instruct-GPTQ", use_fast=True)

### Codellama-34b (GPTQ)

In [ ]:
model_path = "TheBloke/Phind-CodeLlama-34B-v2-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model,device_map="auto", trust_remote_code=True, revision="main")

tokenizer = AutoTokenizer.from_pretrained("TheBloke/Phind-CodeLlama-34B-v2-GPTQ", use_fast=True)

### Wizard-Coder-13b (GPTQ)

In [ ]:
model_path = "TheBloke/WizardCoder-Python-13B-V1.0-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=False,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

### Wizard-Coder-15b (GPTQ)

In [ ]:
model_path = "TheBloke/WizardCoder-15B-1.0-GPTQ"

model = AutoGPTQForCausalLM.from_quantized(model_name_or_path,
        use_safetensors=True,
        device="cuda:0",
        use_triton=False,
        quantize_config=None)

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

pipe = pipeline("text-generation", model=model, tokenizer=tokenizer)

### Phi-2 (GPTQ)

In [ ]:
model_path = "TheBloke/phi-2-GPTQ"

model = AutoModelForCausalLM.from_pretrained(model_name_or_path,
                                             device_map="auto",
                                             trust_remote_code=True,
                                             revision="main")

tokenizer = AutoTokenizer.from_pretrained(model_name_or_path, use_fast=True)

## Functions

### Basics

Basic functions to create the dataset for working with a LLM code generation testing model

In [ ]:
def generate_samples_pass1(problems):

  # Store samples
  samples = []
  n = 0

  # For every problem in dataset
  for task_id, problem in problems.items():
    answer = dict(task_id = task_id, completion = generate_solution_llama13b(problem["prompt"])) # change model name for different model
    samples.append(answer)
    print("step: ", n, "from 163 done")
    n = n+1

  return samples

In [ ]:
def generate_samples_pass10(problems):

  # Store samples
  samples_1 = []
  samples_10 = []
  n = 0
  m = 0
  examples = 1

  # For every problem in dataset
  for task_id, problem in problems.items():
    res = dict(task_id = task_id, completion = generate_solution_llama13b_tmplow(problem["prompt"])) # change model name for different model
    samples_1.append(res)
    for n in range(n, examples):
      answer = dict(task_id = task_id, completion = generate_solution_llama13b_tmphigh(problem["prompt"])) # change model name for different model
      samples_10.append(answer)
      print("step: ", n, "done")
      n = n+1
    n = 0
    print("block done: ", m, "of 163")
    m = m+1

  return (samples_1, samples_10)

### get_code funtions

about: The get_code function family is a group of functions I created to get the fitting result out of every model used.
        My goal was to just get the code as result without prompt or additional components like test or comments.

note: For know just codellama functions are finished and fully useable

In [ ]:
def get_code_llama13b(text: str):

  # Find the index where "def " starts
  begin_index = text.find('def ', 200)
  if begin_index == -1:
      return ""  # return an empty string if "def " is not found

  # Extract the code starting from "def "
  function = text[begin_index:]

  # Find the index of "```" that comes after "def "
  end_code_index = function.find('```')

  # If "```" is found, return the code until just before "```"
  if end_code_index != -1:
      code = function[:end_code_index]
  else:
      # If no "```" is found, return the code until the end of the string
      code = function

  # Remove prompt if existing
  if code.find('"""'):
    promptbegin_index = code.find('"""')
    #print(promptbegin_index)
    code_del = code[promptbegin_index:]
    promptend_index = code_del.find('"""', 10)
    print(promptend_index)
    code_del = code_del[:promptend_index]

    print(code_del)

  # Removing leading and trailing whitespaces and newlines
  return code.strip()

In [6]:
def get_code_wizard15b(text: str):

  # Find the index where "def " starts
  def_index = text.find('def ', 300)
  if def_index == -1:
      return ""  # return an empty string if "def " is not found

  # Extract the code starting from "def "
  code_after_def = text[def_index:]

  # Find the index of "```" that comes after "def "
  end_code_index = code_after_def.find('```')

  # If "```" is found, return the code until just before "```"
  if end_code_index != -1:
      code = code_after_def[:end_code_index]
  else:
      # If no "```" is found, return the code until the end of the string
      code = code_after_def

  # Removing leading and trailing whitespaces and newlines
  return code.strip()

In [ ]:
def get_code_wizard13b(text: str):

  # Find the index where "def " starts
  def_index = text.find('def ', 200)
  if def_index == -1:
      return ""  # return an empty string if "def " is not found

  # Extract the code starting from "def "
  code_after_def = text[def_index:]

  # Find the index of "```" that comes after "def "
  end_code_index = code_after_def.find('# Test')

  # If "```" is found, return the code until just before "```"
  if end_code_index != -1:
      code = code_after_def[:end_code_index]
  else:
      # If no "```" is found, return the code until the end of the string
      code = code_after_def

  # Removing leading and trailing whitespaces and newlines
  return code.strip()

In [ ]:
def get_code_phi2(text: str):

  # Find the index where "def " starts
  def_index = text.find('def ')
  if def_index == -1:
      return ""  # return an empty string if "def " is not found

  # Extract the code starting from "def "
  code_after_def = text[def_index:]

  # Find the index of "```" that comes after "def "
  end_code_index = code_after_def.find('if __name__')

  # If "```" is found, return the code until just before "```"
  if end_code_index != -1:
      code = code_after_def[:end_code_index]
  else:
      # If no "```" is found, return the code until the end of the string
      code = code_after_def

  # Removing leading and trailing whitespaces and newlines
  return code.strip()

### generate_solution functions

about: The generate_solution function family is a group of functions I created to get the fitting result out of every model used.
        My functions take "Human Eval" suggested funtion as build since I will primarily use their test for my research

Credit: "https://github.com/openai/human-eval"

In [ ]:
def generate_solution_llama13b_tmplow(prompt: str):

  # Prompt tempate for code llama models
  prompt_template=f'''[INST] Write Python code to solve the following coding problem that obeys the constraints and passes the example test cases. You must end your code using ```:
  {prompt}
  [/INST]

  '''

  # Prepare input
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

  # Load input to model
  output = model.generate(inputs=input_ids, temperature=0.1, do_sample=True, top_p=0.95, max_new_tokens=512)

  # Decode result
  code = tokenizer.decode(output[0], skip_special_tokens=True)

  # Get only "useable" code
  solution = get_code(code)

  #return code
  return solution

In [ ]:
def generate_solution_llama13b_tmphigh(prompt: str):

  # Prompt tempate for code llama models
  prompt_template=f'''[INST] Write Python code to solve the following coding problem that obeys the constraints and passes the example test cases. You must end your code using ```:
  {prompt}
  [/INST]

  '''

  # Prepare input
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()

  # Load input to model
  output = model.generate(inputs=input_ids, temperature=1, do_sample=True, top_p=0.95, max_new_tokens=512)

  # Decode result
  code = tokenizer.decode(output[0], skip_special_tokens=True)

  # Get only "useable" code
  solution = get_code(code)

  #return code
  return solution

In [ ]:
def generate_solution_wizard15b(prompt: str):

  # Prompt
  prompt_template = '''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction: {prompt}

  ### Response:'''

  # Prombt format
  prompt = prompt_template.format(prompt=prompt)


  # LLM generation
  outputs = pipe(prompt, max_new_tokens=1024, do_sample=True, temperature=0.1, top_k=50, top_p=0.95)


  # Fix output
  code = get_code(outputs[0]['generated_text'])


  return code

In [ ]:
def generate_solution_wizard13b(prompt: str):

  # Prompt
  prompt_template=f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  {prompt}

  ### Response:

  '''


  # LLM generation
  input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.pad_token_id,)


  # Fix output
  code = get_code(tokenizer.decode(output[0]))


  #return code
  return code

In [ ]:
def generate_solution_phi2(prompt: str):

  # Prompt
  #system_message = "You are a code writing assistant."

  # Prompt
  pro=f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
  {prompt}

  ### Response:

  '''

  prompt_template=f'''Instruct: {prompt}
  Output:
  '''

  # LLM generation
  input_ids = tokenizer(pro, return_tensors='pt').input_ids.cuda()
  output = model.generate(inputs=input_ids, temperature=0.2, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)


  # Fix output
  code = get_code(tokenizer.decode(output[0]))


  return code
  #return tokenizer.decode(output[0])

## Calls

The actuall pipeline of calls to produce a dataset for testing

In [ ]:
# Execute pipe
t1 = time.perf_counter()
problems = read_problems()
result = generate_samples_pass10(problems)
json_1 = result[1]
json_10 = result[10]
write_jsonl("LLM_gen_1.jsonl", json_1)
write_jsonl("LLM_gen_10.jsonl", json_10)
t2 = time.perf_counter()
print('time taken to run:',(t2-t1)/60)

step:  0 from 163 done
step:  1 from 163 done
step:  2 from 163 done
step:  3 from 163 done
step:  4 from 163 done
step:  5 from 163 done
step:  6 from 163 done
step:  7 from 163 done
step:  8 from 163 done
step:  9 from 163 done
step:  10 from 163 done
step:  11 from 163 done
step:  12 from 163 done
step:  13 from 163 done
step:  14 from 163 done
step:  15 from 163 done
step:  16 from 163 done
step:  17 from 163 done
step:  18 from 163 done
step:  19 from 163 done
step:  20 from 163 done
step:  21 from 163 done
step:  22 from 163 done
step:  23 from 163 done
step:  24 from 163 done
step:  25 from 163 done
step:  26 from 163 done
step:  27 from 163 done
step:  28 from 163 done
step:  29 from 163 done
step:  30 from 163 done
step:  31 from 163 done
step:  32 from 163 done
step:  33 from 163 done
step:  34 from 163 done
step:  35 from 163 done
step:  36 from 163 done
step:  37 from 163 done
step:  38 from 163 done
step:  39 from 163 done
step:  40 from 163 done
step:  41 from 163 done
st

## Debug

Debug functions for pipeline

In [ ]:
# prompt = "A function that creates an array and fills it with random numbers 1 to 20"
# prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```. Please start the code with a def:
# {prompt}
# [/INST]

# '''

# print("\n\n*** Generate:")

# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.7, do_sample=True, top_p=0.95, top_k=40, max_new_tokens=512)
# print(tokenizer.decode(output[0]))

In [ ]:
# problems = read_problems()
# prob = problems["HumanEval/1"]["prompt"]
# prob = "Write a function that shows the first 100 prime numbers"
# prompt = prob
# prompt_template=f'''[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
# {prompt}
# [/INST]

# '''


In [ ]:
# input_ids = tokenizer(prompt_template, return_tensors='pt').input_ids.cuda()
# output = model.generate(inputs=input_ids, temperature=0.2, do_sample=True, top_p=0.95, max_new_tokens=512)
# code = tokenizer.decode(output[0], skip_special_tokens=True)

In [ ]:
# print(code)

[INST] Write code to solve the following coding problem that obeys the constraints and passes the example test cases. Please wrap your code answer using ```:
Write a function that shows the first 100 prime numbers
[/INST]

[PYTHON]
def get_first_100_prime_numbers(n):
    prime_numbers = []
    for num in range(2, n+1):
        if all(num % i != 0 for i in range(2, num)):
            prime_numbers.append(num)
    return prime_numbers[:100]
[/PYTHON]

[TESTS]
# Test case 1:
assert get_first_100_prime_numbers(100) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97]
# Test case 2:
assert get_first_100_prime_numbers(1000) == [2, 3, 5, 7, 11, 13, 17, 19, 23, 29, 31, 37, 41, 43, 47, 53, 59, 61, 67, 71, 73, 79, 83, 89, 97, 101, 103, 107, 109, 113, 127, 131, 137, 139, 149, 151, 157, 163, 167, 173, 179, 181, 191, 193, 197, 199, 211, 223, 227, 229, 233, 239, 241, 251, 257, 263, 269, 271, 277, 2


In [ ]:
# problems = read_problems()
# prob = problems["HumanEval/0"]["prompt"]

In [ ]:
# res = generate_solution(prob)
# fix = get_code(res)

# print("--------------------------")
# print(res)
# print("--------------------------")
# print(fix)

-1

-1

--------------------------
[INST] Write Python code to solve the following coding problem that obeys the constraints and passes the example test cases. You must end your code using ```:
  from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    """ Check if in given list of numbers, are any two numbers closer to each other than
    given threshold.
    >>> has_close_elements([1.0, 2.0, 3.0], 0.5)
    False
    >>> has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3)
    True
    """

  [/INST]

  

[PYTHON]
from typing import List


def has_close_elements(numbers: List[float], threshold: float) -> bool:
    for i in range(len(numbers) - 1):
        for j in range(i + 1, len(numbers)):
            if abs(numbers[i] - numbers[j]) <= threshold:
                return True
    return False


if __name__ == "__main__":
    print(has_close_elements([1.0, 2.0, 3.0], 0.5))
    print(has_close_elements([1.0, 2.8, 3.0, 4.0, 5.0, 2.0], 0.3))